## Bike Trip Flow Plots in R

In [1]:
options(jupyter.plot_mimetypes = c("text/plain", "image/png"))

In [3]:
library(osmplotr)
library(dodgr)
library(osmdata)
library(ggplot2)
library(magrittr)

In [29]:
# file_list <- c('2016Q3')
file_list <- c('2016Q3','2018Q1','2017Q4','2017Q1','2016Q4','2017Q3','2016Q1','2016Q2','2017Q2','2015Q4','2015Q3','2015Q2')

In [30]:
cols <- colorRampPalette(c("blue", "cyan"))(10)

In [11]:
bb <- get_bbox(c(-75.23, 39.91, -75.10, 40.00))
dat_B <- extract_osm_objects(key="building", bbox=bb)

add_feature() is deprecated; please use add_osm_feature()


In [31]:
for (file in file_list){
    stns <- read.csv('station.csv')
    xy <- stns[, which(names(stns) %in% c("start_lon", "start_lat"))]
    
    path <- paste(file, "flow.csv", sep="-")
    trips <- read.csv(path, row.names = 1)
    trips[is.na(trips)] <- 0
    indx <- match(as.character(stns$start_station), rownames(trips))
    indx <- indx[!is.na(indx)]
    trips <- trips[indx, indx]
    xy <- xy[indx,]
    
    net <- dodgr_streetnet(pts = xy, expand = 0.05) %>% weight_streetnet(wt_profile = "bicycle")
    verts <- dodgr_vertices(net)
    pts <- verts$id[match_pts_to_graph(verts, xy)]

    flow <- dodgr_flows(net, from = pts, to = pts, flow = trips) %>% merge_directed_flows()
    net <- net[flow$edge_id, ]
    net$flow <- flow$flow
    
    map <- osm_basemap(bb, bg = "grey20") %>%
    add_osm_objects(dat_B, col="grey40") %>%
    add_colourbar(zlims = range(net$flow), col = cols)

    map <- map + geom_segment(data = net, size = net$flow/5000, 
                           aes(x = from_lon, y = from_lat, xend = to_lon, yend = to_lat,
                           colour = flow, size = flow)) + 
    scale_colour_gradient(low = "blue", high = "cyan", guide = "none") +
    annotate("text", label = file, x = -75.109, y = 39.995, color = "white")
    mapname = paste(file, "flow.png", sep="-")
    print_osm_map(map, file=mapname)
}